# legacy

## general pythonic xml handling

### get and print all xml tags (in hierarchical order) used in a xml file

In [ ]:
import xml.etree.ElementTree as ET 
from pprint import pprint

In [ ]:
tree = ET.parse('filename')
root = tree.getroot()

elemList = []

for elem in tree.iter():
    elemList.append(elem.tag)

elemList = list(set(elemList))
pprint(elemList)

In [ ]:
for level_1 in root: #write function or enlongen as desired
    print('/Level 1')
    print(level_1)
    for level_2 in level_1:
        print(' / / Level 2:')
        print(level_2)
        for level_3 in level_2:
            print(' / / / Level 3:')
            print(level_3)

# get atomID and graphicURL into .csv from a collection using scandir ✔
- without subdirectories
- only charters with images

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
from pathlib import PurePath

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = '../data/db/mom-data/metadata.charter.public/BischoefeSpeyer/'
extension = ('.cei.xml')

In [ ]:
atomIDs = []
attributes = []

with os.scandir(directoryPath) as it:
    for entry in it:
        if entry.name.endswith(extension) and entry.is_file():
            tree = ET.parse(entry.path)
            root = tree.getroot()
            for tag in root.findall('.//cei:graphic', namespaces):
                atomID = root[0].text
                atomIDs.append(atomID)
                attribute = tag.attrib['url'] 
                attributes.append(attribute)

In [ ]:
img_list = list(zip(atomIDs, attributes))
df = pd.DataFrame(img_list).rename(columns={0: 'atomID', 1: 'url'})
df

In [ ]:
path = PurePath(directoryPath)
pathname = path.name

In [ ]:
df.to_csv(f'../data/output/{pathname}.csv', index=False)

## parsing xml

- findall-method is rather slow
- simple iter should be fastest method but is limited in configuration options
- iterparse is slower than others due to overhead
- simple xpath (lxml) is best since it is both fast and easy to hdle

In [ ]:
#xpath lxml
for file in Path(directoryPath).rglob(fileExtension):
    tree = etree.parse(str(file))
    for tenor in tree.xpath("//cei:tenor", namespaces = namespaces):
        tenor_contents.append(tenor.text)
        atom_IDs.append(tree.xpath("//atom:id/text()", namespaces = namespaces))
        places.append(tree.xpath(".//cei:placeName/text()", namespaces = namespaces))
        languages.append(tree.xpath(".//cei:lang_MOM/text()", namespaces = namespaces))

In [ ]:
#findall etree
for file in Path(directoryPath).rglob(fileExtension):
    root = etree.parse(str(file)).getroot()
    for tenor in root.findall(".//cei:tenor", namespaces):
        tenor_contents.append(etree.tostring(tenor).decode("utf-8")) # diese Zeile killt den Speicher
        atom_IDs.append(root[0].text)
        if root.find('.//cei:placeName', namespaces) is not None:
            places.append(root.find('.//cei:placeName', namespaces).text)
        elif root.find('.//cei:placeName', namespaces) is None:
            places.append(None)
        if root.find('.//cei:lang_MOM', namespaces) is not None:
            languages.append(root.find('.//cei:lang_MOM', namespaces).text)
        elif root.find('.//cei:lang_MOM', namespaces) is None:
            languages.append(None)